# Overview 

We are using pandas to import three data tables to clean and transform them before exporting to Postgres. Each table includes data involving financial information by each state in the U.S., such as housing, income, and poverty rates. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Loading housing data CSV
housing_table = pd.read_csv("Resources/housing_prices.csv")

In [ ]:
# Setting up url for extraction
poverty_url = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_poverty_rate"

In [ ]:
# Setting up url for extraction
income_url = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_income"

# Extract

In [ ]:
# Extracting poverty table from html
table = pd.read_html(poverty_url)

In [ ]:
poverty_table = table[0]

In [ ]:
# Extracting income table from html
table = pd.read_html(income_url)

In [ ]:
income_table = table[1]

# Transform

### Transform - Housing

We transformed our housing price data here.

In [ ]:
# Created dictionary to make transformation process simpler
state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}    

In [ ]:
# Added state code
housing_table['state'] = housing_table['State'].replace(state_codes)

In [ ]:
# Dropped the old state and Zillow columns
housing_dropped_df = housing_table.drop(columns = ["State", "medianValueZillow"])

In [ ]:
# Renamed median price column
housing_rename_df = housing_dropped_df.rename(columns = {'MedianValue': 'median_home_price'})

In [ ]:
# Set state code as index
housing_df = housing_rename_df.set_index('state')

### Transform - Poverty

Next, we transformed our poverty data by state.

In [ ]:
# Added state code
poverty_table['state'] = poverty_table['State or Territory'].replace(state_codes)

In [ ]:
# Dropped old state labels and raw population total columns
poverty_dropped_df = poverty_table.drop(columns = ["State or Territory", "Population", "Population underPoverty Line"])

In [ ]:
# Dropped rows that were null or incongruent (territories, U.S. total)
poverty_dropped_rows_df = poverty_dropped_df.drop(axis = 0, index = [0, 1, 9, 53])

In [ ]:
# Renamed poverty rate column
poverty_rename_df = poverty_dropped_rows_df.rename(columns = {'Poverty Rate': 'poverty_rate'})

In [ ]:
# Set state code as index
poverty_df = poverty_rename_df.set_index('state')

### Transform - Income

Finally, we transformed the income data by state.

In [ ]:
# Added state code
income_table['state'] = income_table['States and Washington, D.C.'].replace(state_codes)

In [ ]:
# Renamed column with messy name
income_aagr_df = income_table.rename(columns = {income_table.columns[12]: 'avg_income_growth_rate'})

In [ ]:
# Made a copy dataframe with columns we wanted to keep
income_dropped_df = income_aagr_df[['state', '2019', 'avg_income_growth_rate']].copy()

In [ ]:
# Dropped rows that were not needed
income_dropped_rows_df = income_dropped_df.drop(axis = 0, index = [0, 1])

In [ ]:
# Renamed column
income_rename_df = income_dropped_rows_df.rename(columns = {'2019': 'median_income'})

In [ ]:
# Formatted median incomes to integers without $ or ,
income_rename_df['median_income'] = income_rename_df['median_income'].apply(lambda x: x.replace('$', '')).apply(lambda x: x.replace(',', '')).astype(np.int64)

In [ ]:
# Set state code as index
income_df = income_rename_df.set_index('state')